# NCAA tournament data - webscraping

In [ ]:
#imports for webscraping
from bs4 import BeautifulSoup
from urllib.request import urlopen
#for saving and load data files
import pickle

In [2]:
#objects to store data
tournamentGameData = {} #dictionary with year as key and value is a list of lists containing [team1,team2,overall score]
#example: '2000': [['Duke', 'Lamar', 137], ['Kansas', 'DePaul', 158]]

teamYearData = {} #dictionary with year as key and value is a dictionary with team as key and value in the form of [[team stats],[opponent stats]] 
#team/oppenent stats: [G, MP, FG, FGA, FG%, 2P, 2PA, 2P%, 3P, 3PA, 3P%, FT, FTA, FT%, ORB, DRB, TRB, AST, STL, BLK, TOV, PF, PTS, PTS/G]
#example: '2000': {'Duke': [[34, '', 1045, 2172, 0.481, 761, 1430, 0.532, 284, 742, 0.383, 618, 833, 0.742, 453, 860, 1313, 584, 333, 191, 480, 552, 2992, 88.0], [34, '', 934, 2238, 0.417, 737, 1686, 0.437, 197, 552, 0.357, 360, 537, 0.67, 526, 757, 1283, 472, 240, 123, 607, 690, 2425, 71.3]]}

In [ ]:
#webscraping has already been run and the data is saved in files: tournamentGameData.pickle, teamYearData.pickle
#open dicts from file

pickle_in1 = open("tournamentGameData_real.pickle","rb")
tournamentGameData = pickle.load(pickle_in1)

pickle_in2 = open("teamYearData_real.pickle","rb")
teamYearData = pickle.load(pickle_in2)

In [ ]:
#webscraping done here
#runtime about a minute per year
for i in range(2000,2018):
    year = str(i)
    print(year)
    tournamentGameData[year] = []
    teamYearData[year] = {}
    
    url ='https://www.sports-reference.com/cbb/postseason/'+year+'-ncaa.html'
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    numSchoolsAdded = 1
    for tround in soup.findAll('div', attrs={'class':'round'}):
        for game in tround.findAll('div',recursive=False):
            gameDetails = game.findAll('a')
            if len(gameDetails) > 3:
                schools = [gameDetails[0].text.strip(),gameDetails[2].text.strip()]
                gameData = [schools[0],schools[1],int(gameDetails[1].text.strip())+int(gameDetails[3].text.strip())]
                tournamentGameData[year].append(gameData)
                index = 0
                for school in schools:
                    if school not in teamYearData[year]:
                        teamurl ='https://www.sports-reference.com'+gameDetails[index]["href"]
                        teamhtml = urlopen(teamurl)
                        teamsoup = BeautifulSoup(teamhtml, 'html.parser')
                        table = teamsoup.find('table', attrs={'id':'team_stats'}).find('tbody')
                        teamStats = []
                        for row in table.findAll('tr'):
                            temp = []
                            if row.has_attr("class"):
                                continue
                            for stat in row.findAll('td'):
                                temp.append(stat.text.strip())
                            teamStats.append(temp)
                        teamYearData[year][school] = teamStats
                        print("{}, ".format(numSchoolsAdded),end = "")
                        numSchoolsAdded += 1
                        index = index + 2
    print("done")        
        
print("done")
        

In [ ]:
#convert data from string to float/int, empty entries represented with ''
for i in range(2000,2018):
    year = str(i)
    for team in teamYearData[year]:
        newStats = []
        for stats in teamYearData[year][team]:
            index = 0
            newStat = []
            for stat in stats:
                if stat != '':
                    if index in [4,7,10,13,23]:
                        newStat.append(float(stat))
                    else:
                        newStat.append(int(stat)) 
                else:
                    newStat.append('')
                index +=1
            newStats.append(newStat)
        teamYearData[year][team] = newStats

In [ ]:
#save dicts to file
import pickle

pickle_out1 = open("tournamentGameData.pickle","wb")
pickle.dump(tournamentGameData, pickle_out1)
pickle_out1.close()

pickle_out2 = open("teamYearData.pickle","wb")
pickle.dump(teamYearData, pickle_out2)
pickle_out2.close()

#how to load
#pickle_in = open("dict.pickle","rb")
#example_dict = pickle.load(pickle_in)